In [1]:
import tensorflow as tf
import glob
import collections
import numpy as np
import random
import math
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
import os
from tensorflow.contrib.tensorboard.plugins import projector



import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector


In [2]:
current_path = os.path.dirname(os.path.realpath(sys.argv[0]))

parser = argparse.ArgumentParser()
parser.add_argument(
    '--log_dir',
    type=str,
    default=os.path.join(current_path, 'log-tb-4-june'),
    help='The log directory for TensorBoard summaries.')
FLAGS, unparsed = parser.parse_known_args()

# Create the directory for TensorBoard variables if there is not.
if not os.path.exists(FLAGS.log_dir):
    os.makedirs(FLAGS.log_dir)

In [3]:
# paths = glob.glob('MorpedTextedWWO/*.txt')
paths = glob.glob('/home/parth/work_files/oxy-4june/www-plaintext/*.txt')

In [4]:
newFullText  = ""

In [5]:
for path in paths:
    with open(path, "r") as f:
        newFullText += f.read().split("\n")[1] 

In [6]:
newFullText = newFullText.lower()

In [7]:
vocabulary_size = 50000

In [8]:
words_all = newFullText.split(" ")
# words_all = [w for w in words_all if w not in stop_words.ENGLISH_STOP_WORDS]

In [9]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [10]:
data, count, dictionary, reverse_dictionary = build_dataset(words_all, vocabulary_size)

In [11]:
def createBatch(size, num_skips, skip_window):
    global data_index
    batch = np.ndarray(shape=(size), dtype=np.int32)
    labels = np.ndarray(shape=(size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span) # pylint: disable=redefined-builtin    
    
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    
    
#     print("******************")
#     print(buffer)
#     print(len(buffer))
#     print("******************")
    
    
    for i in range(size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
            
            
#             print("index", buffer[skip_window], buffer[context_word])
#             print("label", skip_window, buffer[skip_window])

        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [12]:
batch_size = 144
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 12  # How many words to consider left and right.
num_skips = 12  # How many times to reuse an input to generate a label.
num_sampled = 32  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [13]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
        nce_weights = tf.Variable(
            tf.truncated_normal(
                [vocabulary_size, embedding_size],
                stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
# num_steps = 100001
# data_index = 0
# # for step in xrange(num_steps):
# #     batch_inputs, batch_labels = createBatch(batch_size, num_skips,skip_window)


In [15]:
# Step 5: Begin training.
data_index = 0
num_steps = 5000001
# num_steps = 1000001

with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter(FLAGS.log_dir, session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = createBatch(batch_size, num_skips,
                                                skip_window)
#         print(batch_inputs)
#         print(batch_labels)
#         print("**************************")
        
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
            [optimizer, merged, loss],
            feed_dict=feed_dict,
            run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)
        
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

    final_embeddings = normalized_embeddings.eval()

    # Write corresponding labels for the embeddings.
    with open(FLAGS.log_dir + '/metadata.tsv', 'w') as f:
        for i in xrange(vocabulary_size):
            f.write(reverse_dictionary[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join(FLAGS.log_dir, 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join(FLAGS.log_dir, 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

writer.close()

Initialized
Average loss at step  0 :  159.7100372314453
Average loss at step  2000 :  75.73207667636872
Average loss at step  4000 :  44.24920387470722
Average loss at step  6000 :  40.730657796144484
Average loss at step  8000 :  29.7687312605381
Average loss at step  10000 :  22.3921632540226
Average loss at step  12000 :  22.950083320736884
Average loss at step  14000 :  19.369962785482407
Average loss at step  16000 :  16.800360976099967
Average loss at step  18000 :  14.167467841744424
Average loss at step  20000 :  12.653556562304496
Average loss at step  22000 :  14.23383607852459
Average loss at step  24000 :  11.842587855458259
Average loss at step  26000 :  10.93060578083992
Average loss at step  28000 :  8.508837864398956
Average loss at step  30000 :  8.953792636871338
Average loss at step  32000 :  8.465025626540184
Average loss at step  34000 :  8.310250946044922
Average loss at step  36000 :  7.994609128594399
Average loss at step  38000 :  7.730327393651009
Average los

Average loss at step  330000 :  4.474761876106262
Average loss at step  332000 :  4.543403644323349
Average loss at step  334000 :  4.603809056997299
Average loss at step  336000 :  4.475053770542145
Average loss at step  338000 :  4.385532939434052
Average loss at step  340000 :  4.351508023262024
Average loss at step  342000 :  4.3140484303236
Average loss at step  344000 :  4.275401655077935
Average loss at step  346000 :  4.296652863502502
Average loss at step  348000 :  4.252118652582168
Average loss at step  350000 :  4.262676032066345
Average loss at step  352000 :  4.264108455300331
Average loss at step  354000 :  4.276390666365623
Average loss at step  356000 :  4.235719795227051
Average loss at step  358000 :  4.236018574357033
Average loss at step  360000 :  4.25402494931221
Average loss at step  362000 :  4.434459977388382
Average loss at step  364000 :  4.401349250435829
Average loss at step  366000 :  4.171463490962982
Average loss at step  368000 :  4.123364877581596
Ave

Average loss at step  660000 :  4.249952207684517
Average loss at step  662000 :  4.305019678354263
Average loss at step  664000 :  4.32931339263916
Average loss at step  666000 :  4.318633353233337
Average loss at step  668000 :  4.3252765306234355
Average loss at step  670000 :  4.265398904442788
Average loss at step  672000 :  4.346717289090156
Average loss at step  674000 :  4.267895581305027
Average loss at step  676000 :  4.179857656240463
Average loss at step  678000 :  4.023809662103653
Average loss at step  680000 :  4.0627296621799465
Average loss at step  682000 :  3.961513548016548
Average loss at step  684000 :  4.122569019913674
Average loss at step  686000 :  8.161204714387654
Average loss at step  688000 :  4.370270961999894
Average loss at step  690000 :  4.259252187728882
Average loss at step  692000 :  4.429004257380963
Average loss at step  694000 :  4.355642389833927
Average loss at step  696000 :  4.371821584224701
Average loss at step  698000 :  4.287897708773613

Average loss at step  988000 :  4.275804115891456
Average loss at step  990000 :  4.287743666529655
Average loss at step  992000 :  4.278466325879097
Average loss at step  994000 :  4.200004235029221
Average loss at step  996000 :  4.294155029892922
Average loss at step  998000 :  4.2209870955348014
Average loss at step  1000000 :  4.321140387654305
Average loss at step  1002000 :  4.323150376677513
Average loss at step  1004000 :  4.2862759510278705
Average loss at step  1006000 :  4.27425387096405
Average loss at step  1008000 :  4.262414734959602
Average loss at step  1010000 :  4.264902447223664
Average loss at step  1012000 :  4.233947203040123
Average loss at step  1014000 :  4.161872918486595
Average loss at step  1016000 :  4.242193931221962
Average loss at step  1018000 :  4.220278121232987
Average loss at step  1020000 :  4.165673514485359
Average loss at step  1022000 :  4.23076765525341
Average loss at step  1024000 :  4.205060137152672
Average loss at step  1026000 :  4.05

Average loss at step  1310000 :  4.0577375831604
Average loss at step  1312000 :  3.9974496145248413
Average loss at step  1314000 :  4.0703355870246885
Average loss at step  1316000 :  4.21774648976326
Average loss at step  1318000 :  4.162503222048283
Average loss at step  1320000 :  4.1034983917474745
Average loss at step  1322000 :  4.034387312412262
Average loss at step  1324000 :  4.1784110455513
Average loss at step  1326000 :  4.127821078419685
Average loss at step  1328000 :  4.294067766070366
Average loss at step  1330000 :  4.192228624820709
Average loss at step  1332000 :  4.317530919551849
Average loss at step  1334000 :  4.333945018410683
Average loss at step  1336000 :  4.206474745094776
Average loss at step  1338000 :  4.284981201648712
Average loss at step  1340000 :  4.2339026602506635
Average loss at step  1342000 :  4.176721128702163
Average loss at step  1344000 :  4.142412091732025
Average loss at step  1346000 :  4.139553427815438
Average loss at step  1348000 : 

...

Average loss at step  3564000 :  4.0770897070169445
Average loss at step  3566000 :  4.152059184253216
Average loss at step  3568000 :  4.214602373361587
Average loss at step  3570000 :  4.327242448925972
Average loss at step  3572000 :  4.2528468831777575
Average loss at step  3574000 :  4.32986822283268
Average loss at step  3576000 :  4.2931571506261825
Average loss at step  3578000 :  4.2346339802742
Average loss at step  3580000 :  4.22492806661129
Average loss at step  3582000 :  4.177823325157165
Average loss at step  3584000 :  4.2161992925405505
Average loss at step  3586000 :  4.482038970708847
Average loss at step  3588000 :  4.294186702489853
Average loss at step  3590000 :  4.180629605889321
Average loss at step  3592000 :  4.11440002477169
Average loss at step  3594000 :  4.103549919009208
Average loss at step  3596000 :  4.115594978451729
Average loss at step  3598000 :  4.216429683923721
Average loss at step  3600000 :  4.275433346390725
Average loss at step  3602000 : 

Average loss at step  3886000 :  3.959756120085716
Average loss at step  3888000 :  4.220717472195625
Average loss at step  3890000 :  4.435142289400101
Average loss at step  3892000 :  4.258772900342941
Average loss at step  3894000 :  4.220959267556667
Average loss at step  3896000 :  4.103786422967911
Average loss at step  3898000 :  4.02489415872097
Average loss at step  3900000 :  4.216237216591835
Average loss at step  3902000 :  4.158270512580872
Average loss at step  3904000 :  4.185557514190674
Average loss at step  3906000 :  4.117013378858566
Average loss at step  3908000 :  4.11818995654583
Average loss at step  3910000 :  4.264666528344154
Average loss at step  3912000 :  4.17735260272026
Average loss at step  3914000 :  4.172715906262398
Average loss at step  3916000 :  4.175023621082306
Average loss at step  3918000 :  4.17156510579586
Average loss at step  3920000 :  4.104652265071869
Average loss at step  3922000 :  4.181418460488319
Average loss at step  3924000 :  4.

Average loss at step  4208000 :  4.18959337925911
Average loss at step  4210000 :  4.375190306723118
Average loss at step  4212000 :  4.230963682651519
Average loss at step  4214000 :  4.216729292154312
Average loss at step  4216000 :  4.280466813325882
Average loss at step  4218000 :  4.261986058831215
Average loss at step  4220000 :  4.190560103058815
Average loss at step  4222000 :  4.155306111931801
Average loss at step  4224000 :  4.226473331809044
Average loss at step  4226000 :  4.215585485816002
Average loss at step  4228000 :  4.1666925798654555
Average loss at step  4230000 :  4.119848558306694
Average loss at step  4232000 :  4.148442395567894
Average loss at step  4234000 :  4.141288810610771
Average loss at step  4236000 :  4.030060844838619
Average loss at step  4238000 :  4.035449810624122
Average loss at step  4240000 :  4.200327381014824
Average loss at step  4242000 :  4.137929275155067
Average loss at step  4244000 :  4.044119675159454
Average loss at step  4246000 :

Average loss at step  4530000 :  4.149752880573272
Average loss at step  4532000 :  4.1846891183853145
Average loss at step  4534000 :  4.210635807871818
Average loss at step  4536000 :  4.168481040239334
Average loss at step  4538000 :  4.087898511469364
Average loss at step  4540000 :  3.9526787291765215
Average loss at step  4542000 :  4.253226108193397
Average loss at step  4544000 :  4.150192819237709
Average loss at step  4546000 :  4.24141846370697
Average loss at step  4548000 :  4.281536524534226
Average loss at step  4550000 :  4.183938649535179
Average loss at step  4552000 :  4.194679790616036
Average loss at step  4554000 :  4.132886512160301
Average loss at step  4556000 :  4.102761484861374
Average loss at step  4558000 :  4.043122527241707
Average loss at step  4560000 :  4.090080385327339
Average loss at step  4562000 :  4.052859813511372
Average loss at step  4564000 :  4.242203338384629
Average loss at step  4566000 :  4.193543463706971
Average loss at step  4568000 

Average loss at step  4852000 :  4.080639743447303
Average loss at step  4854000 :  4.074258532762528
Average loss at step  4856000 :  4.051882431149482
Average loss at step  4858000 :  4.05602584028244
Average loss at step  4860000 :  4.051699365735054
Average loss at step  4862000 :  4.044718139529229
Average loss at step  4864000 :  4.043008219718933
Average loss at step  4866000 :  4.080620696663856
Average loss at step  4868000 :  4.039088836669922
Average loss at step  4870000 :  4.053397094011307
Average loss at step  4872000 :  4.043773194551468
Average loss at step  4874000 :  4.230286183476448
Average loss at step  4876000 :  4.053557598471642
Average loss at step  4878000 :  3.9239590253829957
Average loss at step  4880000 :  3.9110835117101668
Average loss at step  4882000 :  4.092734337806702
Average loss at step  4884000 :  4.1930430165827275
Average loss at step  4886000 :  4.153760343670845
Average loss at step  4888000 :  4.187362653255462
Average loss at step  4890000

In [16]:
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')

    plt.savefig(filename)


try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(
        perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))

except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)


In [17]:
FLAGS.log_dir

'/home/parth/tf/lib/python3.5/site-packages/log-tb-4-june'

In [18]:
def top_n_words(word, final_embeddings, k=10):
    word_id = dictionary[word]
    similarities = cosine_similarity([final_embeddings[word_id]], final_embeddings)[0]
    sort = np.argsort(similarities)[::-1]
    probs = []
    for x in sort[0:k]:
        probs.append((similarities[x], reverse_dictionary[x]))
    return probs

In [19]:
def top_n_words_close(word, word2, final_embeddings, k=10):
    word_id = dictionary[word]
    word2_id = dictionary[word2]
    similarities = cosine_similarity([final_embeddings[word_id]], [final_embeddings[word2_id]])[0]
#     sort = np.argsort(similarities)[::-1]
#     probs = []
#     for x in sort[0:k]:
#         probs.append((similarities[x], reverse_dictionary[x]))
    return similarities[0]

In [20]:
word_id = dictionary["coquette"]
print(count[word_id])

('coquette', 19)


In [21]:
top_n_words_close("true","false", final_embeddings, k=10)

0.59260124

In [26]:
top_n_words("grace", final_embeddings, k=20)

[(1.0, 'grace'),
 (0.77568376, 'grace,'),
 (0.6773015, 'heavenly'),
 (0.6656777, 'perfect'),
 (0.64709115, 'graces'),
 (0.6425134, 'gift'),
 (0.62905186, 'favour'),
 (0.6281699, 'hand'),
 (0.6245138, 'beauty,'),
 (0.6233004, 'grant'),
 (0.6213466, 'face.'),
 (0.6208815, 'smiling'),
 (0.6200605, 'presence'),
 (0.61119705, 'beauties'),
 (0.6029942, 'true'),
 (0.60263026, 'glory'),
 (0.6017875, 'blessed'),
 (0.6001467, 'see,'),
 (0.59968996, 'shew'),
 (0.5996439, 'honour')]